In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
crime_df_path = 'data/Crimes_-_2001_to_Present.csv'
crime_df = pd.read_csv(crime_df_path)

In [52]:
crime_df.shape

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'day_date', 'year'],
      dtype='object')

In [5]:
crime_df['Date'] = pd.to_datetime(crime_df['Date'], format = '%m/%d/%Y %I:%M:%S %p')
crime_df['day_date'] = crime_df['Date'].dt.date

Subset the 2016-2019 data

In [41]:
crime_df_16_19 = crime_df[crime_df.day_date > datetime.date(2015, 12, 31)]
crime_df_16_19 = crime_df_16_19[crime_df_16_19.day_date < datetime.date(2020, 1, 1)]
min_date, max_date = crime_df_16_19.day_date.min(), crime_df_16_19.day_date.max()
print(f'Min date: {min_date} and Max date: {max_date}')

Min date: 2016-01-01 and Max date: 2019-12-31


Features Engineering

In [50]:
#rename for ease
cdf = crime_df_16_19

In [55]:
print(cdf.shape)
print(cdf.columns)
cdf.dtypes

(1067294, 23)
Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'day_date'],
      dtype='object')


ID                               int64
Case Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                             int64
District                       float64
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   float64
Y Coordinate                   float64
Year                             int64
Updated On                      object
Latitude                       float64
Longitude                      float64
Location                        object
day_date                        object
dtype: object

In [128]:
cdf_beats = pd.DataFrame(cdf.groupby(['Year', 'Beat'])['ID'].count())
cdf_beats.rename(columns={'ID': 'record_cnt'}, inplace=True)
cdf_beats

record_cnt
Year Beat            
2016 111         1843
     112         1849
     113         1085
     114         1049
     121          810
...               ...
2019 2531         690
     2532        1026
     2533        1604
     2534        1192
     2535         838

[1096 rows x 1 columns]

In [129]:
print(cdf['Primary Type'].value_counts())
cdf['Primary Type'].value_counts().index

THEFT                                253701
BATTERY                              198846
CRIMINAL DAMAGE                      114565
ASSAULT                               79067
DECEPTIVE PRACTICE                    76996
OTHER OFFENSE                         68571
NARCOTICS                             53644
BURGLARY                              48669
MOTOR VEHICLE THEFT                   41622
ROBBERY                               41511
CRIMINAL TRESPASS                     26846
WEAPONS VIOLATION                     19931
OFFENSE INVOLVING CHILDREN             9369
PUBLIC PEACE VIOLATION                 5997
CRIM SEXUAL ASSAULT                    5441
INTERFERENCE WITH PUBLIC OFFICER       4875
SEX OFFENSE                            4539
PROSTITUTION                           2933
HOMICIDE                               2566
ARSON                                  1708
CRIMINAL SEXUAL ASSAULT                1097
LIQUOR LAW VIOLATION                    918
STALKING                        

Index(['THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'ASSAULT', 'DECEPTIVE PRACTICE',
       'OTHER OFFENSE', 'NARCOTICS', 'BURGLARY', 'MOTOR VEHICLE THEFT',
       'ROBBERY', 'CRIMINAL TRESPASS', 'WEAPONS VIOLATION',
       'OFFENSE INVOLVING CHILDREN', 'PUBLIC PEACE VIOLATION',
       'CRIM SEXUAL ASSAULT', 'INTERFERENCE WITH PUBLIC OFFICER',
       'SEX OFFENSE', 'PROSTITUTION', 'HOMICIDE', 'ARSON',
       'CRIMINAL SEXUAL ASSAULT', 'LIQUOR LAW VIOLATION', 'STALKING',
       'KIDNAPPING', 'GAMBLING', 'INTIMIDATION',
       'CONCEALED CARRY LICENSE VIOLATION', 'OBSCENITY', 'NON-CRIMINAL',
       'HUMAN TRAFFICKING', 'PUBLIC INDECENCY', 'OTHER NARCOTIC VIOLATION',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'NON - CRIMINAL'],
      dtype='object')

In [131]:
#Adding primary type counts to each beat-year
cdf_beats.reset_index(inplace=True)
by_primary = pd.DataFrame(cdf.groupby(['Year', 'Beat', 'Primary Type'])['ID'].count())
by_primary.reset_index(inplace=True)
primaries = list(cdf['Primary Type'].value_counts().index)
for primary in primaries:
    filtered = by_primary[by_primary['Primary Type'] == primary]
    filtered.drop('Primary Type', axis =1, inplace=True)
    filtered.rename(columns={'ID': primary}, inplace=True)
    cdf_beats = cdf_beats.merge(filtered, how='left', on=['Year', 'Beat'])


cdf_beats

/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Year,Beat,record_cnt,THEFT,BATTERY,CRIMINAL DAMAGE,ASSAULT,DECEPTIVE PRACTICE,OTHER OFFENSE,NARCOTICS,...,GAMBLING,INTIMIDATION,CONCEALED CARRY LICENSE VIOLATION,OBSCENITY,NON-CRIMINAL,HUMAN TRAFFICKING,PUBLIC INDECENCY,OTHER NARCOTIC VIOLATION,NON-CRIMINAL (SUBJECT SPECIFIED),NON - CRIMINAL
0,2016,111,1843,1143,116,57.0,84,222.0,37,17.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,112,1849,1115,123,41.0,55,318.0,44,5.0,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2016,113,1085,482,151,58.0,46,149.0,21,24.0,...,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN
3,2016,114,1049,558,93,53.0,36,132.0,32,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,121,810,349,63,63.0,27,118.0,46,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2019,2531,690,108,158,71.0,62,67.0,58,43.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,2019,2532,1026,148,237,104.0,78,82.0,92,74.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,2019,2533,1604,503,241,137.0,104,120.0,65,96.0,...,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1094,2019,2534,1192,189,270,149.0,118,44.0,72,68.0,...,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
